# **MDX 데이터 분석하기**
https://pypi.org/project/mdict-utils/
1. **MDX 사전파일을** 활용하여 객체에 구분하기
1. 각 사전의 첫번째 의미를 통해서 **NER** 사전 만들기
1. 추후에 보완을 해서 wordnet 으로 강화하기

## **1 레시피 SQlite 데이터 불러오기**
작업한 내용을 대상으로 중분류 내용 추가하기
- 법률, 식품, 자동차 등 중분류 내용이 당장은 필요가 없어 보인다.
- 작업을 진행하면서 필요하면 추가하기

```python
import pickle
with open('data/nerDict.pk', 'wb') as handle:
    pickle.dump(nerDict, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('data/nerDict.pk', 'rb') as handle:
    nerDict = pickle.load(handle)
nerDict.shape
```

In [1]:
import sqlite3
import pandas as pd
def mdxdb_to_df(file):
    conn = sqlite3.connect(file)
    resp = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tableNames = [name[0]  for name in resp]
    return pd.read_sql_query("SELECT * FROM {}".format(tableNames[1]), conn)

# 레시피 데이터베이스 불러오기
df    = mdxdb_to_df('backup/recipe.db')
menus = df.entry.values.tolist()
len(menus), " ".join(menus)[:200]

(908,
 '가오리조림 가오리찜 가자미야채탕 가자미찜 가지냉채 가지불고기 가지선 가지오징어냉채 가지조림 가지회 각색두부전골 각색버섯덮밥 간장 갈비구이 갈비구이찜 갈비찜 갈비탕 감자가자미구이 감자갈비탕 감자고로케 감자그라땡 감자대구오븐구이 감자두부조림 감자막튀김 감자맑은국 감자밥 감자베이컨볶음 감자보트샐러드 감자볶음 감자볶음밥 감자볶음밥치즈구이 감자부침 감자새우그라땡 감')

## **2 식재료 관련 데이터 찾기**
작업한 내용을 대상으로 중분류 내용 추가하기
1. **식품 관련 내용** 이 설명에 포함된 경우들을 찾아서 정리하기
1.[전은경  맛을 표현하는 단어 (2003)](https://m.blog.naver.com/PostView.nhn?blogId=seongho0805&logNo=150048557298&proxyReferer=https%3A%2F%2Fwww.google.com%2F)
1. 설명 내용에 **"식용", "양념" , "음식", "고기", "야채", "열매", "잎", "줄기", "(식물)", "(동물)"** 포함내용 찾기

```python
import xlrd
xls = xlrd.open_workbook('backup/foodOpenData.xls', on_demand=True)
sheetName = xls.sheet_names()
df = pd.read_excel('backup/foodOpenData.xls', sheetName=sheetName)

import pickle
with open('data/nerDict.pk', 'wb') as handle:
    pickle.dump(nerDict, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('data/nerDict.pk', 'rb') as handle:
    nerDict = pickle.load(handle)
```

In [6]:
import pickle
with open('data/nerDict.pk', 'rb') as handle:
    nerDict = pickle.load(handle)
nerDict.shape

(388091, 2)

In [3]:
food_unique_token = ["식용","양념","음식","물고기","고기","동물","야채","열매","잎","줄기","생선"]

# 식품에 해당하면 결과를 출력
from konlpy.tag import Okt
def check_food_noun(query):
    q_data = nerDict[nerDict.Text == query].Data.values.tolist()
    q_data = [_[0][1]   for _ in q_data]      # 대표개념만 검색
    for no, word in enumerate(q_data):
        for _ in word:
            if len(_) > 5:                    # 너무 짧으면 제외
                if _.find("¶") == -1:         # 예문은 제외
                    t_noun = Okt().nouns(_)   # Okt() 명사추출
                    print(t_noun)
                    for check_noun in t_noun: # 추출 단어의 확인
                        if check_noun in food_unique_token:
                            return query   # print("식품에 해당")

check_food_noun("식빵")

['밀가루', '효모', '반죽', '식용', '빵', '식', '면포']


'식빵'

In [51]:
# 원문 내용의 확인
query     = "바이니"
q_data    = nerDict[nerDict.Text == query].Data.values.tolist()
q_data
# itemIndex = nerDict.Text.values.tolist()
# itemData  = nerDict.Data.values.tolist()

[]

# **메뉴 데이터 N-Gram 분석**

## **1 N-Gram 데이터**
**itemIndexTemp** 를 활용하며 고유단어 찾기

In [5]:
import pandas as pd
menus = pd.read_csv('data/muyong_menus.csv', encoding='ms949')
menus = menus.dropna(subset=['메인1', '메인2'])
menus = menus.fillna('')  # NaN 값을 지운다

# DataFrame 에서 고유 Token List 추출
tokens, result = [], []
for _ in menus.columns[4:]:
    tokens += menus[_].values.tolist()

# &, %, ., / 등의 제거 후, 고유 Token 문서 만들기 
import re
for _ in tokens:
    _temp = re.findall(r"\w+", _)
    if _temp:  result += _temp
tokens = list(set(result))
print(len(tokens)), " ".join(tokens[:25])

2903


(None,
 '줄감자튀김 후랑크브로콜리칠리볶음 파인소스 베이컨김치숙주전 추억의엣날도시락 가자미칠리소스 매콤소고기두부조림 실치땅콤볶음 진미채야채무침 수제돈까스 맛살팽이전 단호박닭찜 느타리브로컬리볶음 오이스틱 매운등갈비떡찜 직화산적 칼집비엔나브로콜리볶음 올방개묵 제육숙주굴소스볶음 달래장 매콤군만두 얼큰콩나물무침 청양풍돈사태떡찜 수제돈깐풍 매콤오징어떡깻순볶음')

In [17]:
# Okt 모듈로 명사만 찾기 (대중적 선별기준)
from konlpy.tag import Okt, Hannanum, Mecab
token_Okt = Okt().nouns(" ".join(tokens))
len(set(token_Okt)), len(token_Okt)

(900, 7680)

In [21]:
token_Okt = Mecab().nouns(" ".join(tokens))
len(set(token_Okt)), len(token_Okt)

(961, 7556)

In [19]:
token_Okt = Hannanum().nouns(" ".join(tokens))
len(set(token_Okt)), len(token_Okt)

(2888, 2894)

In [ ]:
# Today....

In [ ]:
# 결과 중 사전에 포함된 단어와, 비포함 단어들로 필터링 진행하기

# **NER 사전 수집결과의 해석**
자동화 알고리즘 및 구조화를 활용하여 모델링을 보완하기

In [10]:
# 30만개 중 11만개의 식품 연관단어의 추출
with open('data/food_nouns.txt', 'r') as f:
    token_nouns = f.read()

token_nouns = sorted(set(token_nouns.split(',')))
len(token_nouns)

11640

In [8]:
# 원문 내용의 확인
query     = "가는개여뀌"
q_data    = nerDict[nerDict.Text == query].Data.values.tolist()
q_data
# itemIndex = nerDict.Text.values.tolist()
# itemData  = nerDict.Data.values.tolist()

[[('명사',
   ['명사',
    '『식물』',
    '마디풀과의 한해살이풀. 높이는 30~50cm이며, 잎은 어긋나고 피침 모양이다. 7~8월에 붉은색이 도는 꽃이 수상(穗狀) 화서로 피고 열매는 수과(痩果)를 맺는다. 물속에서 자라는데 한국의 경기ㆍ강원ㆍ경북ㆍ경남, 일본, 중국 등지에 분포한다. ≒붉은대동여뀌. (persicaria trigonocarpa)\r\n'])]]

In [28]:
" ".join(sorted(set(token_Okt)))

'가래떡 가루 가슴살 가쓰 가쓰오 가오리 가자미 가자미조림 가지 간 간장 간장지 갈릭 갈비 갈비구이 갈비찜 갈비탕 갈치구이 갈치조림 감귤 감자 감자비 감자수 감자알 감자엿 감자전 감자조림 감자채 감자탕 감자튀김 갓김치 강정 강정무 강회 개해 거지 거트 건 건새 건포도 검정깨 겉절이 게 게맛살 겨자 겨자채 견과 겹 계란 계란구이 계란장조림 계란찜 계장 계탕 고갈 고구마 고구마튀김 고기 고기볶음 고기전골 고등어 고등어구이 고등어자반 고등어조림 고로케 고사리 고사리나물 고추 고추쌈 고추알 고추잡채 고추장 고추장찌개 고추전 고춧잎 곤약 골 골맹이 골뱅이 곰피 곱창 과일 교자 교촌 구 구이 국 국내산 국물 국밥 국산 국수 군 군만두 굴 굴비 궁중 그 그릇 근대 근대나물 글 기민 기산 기완 기타 기파 김 김구이 김밥 김치 김치볶음밥 김치전 김치찌개 김치홍 깁 까르보 까스 깍 깍두기 깐 깐풍기 깨 깨소 깻잎 깻잎나물 깻잎쌈 꼬마 꼬미 꼬치 꼬치전 꽁치 꽃 꽃게 꽈리 꽈리고추 끼 나 나떡 나라 나물 나물밥 나물볶음 나후 낙지 난 날 냉국 냉열 냉이 냉채 너 너겟 너비 너비아니 네비 노각 노미야 녹두 녹두전 누드 누룽지 느타리 느타리버섯 닝 다대기 다리 다시마 다시마튀각 단감 단무지 단전 단호박 달래장 닭 닭갈비 닭강정 닭개장 닭곰탕 닭꼬치 닭도리탕 닭백숙 닭볶음탕 닭살 닭조림 닭찜 닭튀김 당근 당면 대 대구 대구구이 대국 대왕 덮밥 데 데리 데미 도너츠 도라 도라지 도라지나물 도라지생채 도로리 도시락 도토리 도토리묵 돈 돈가스 돈나물 돈육 돗나물 동 동그랑땡 동양 동태 동태찌개 돼지 된장 된장국 된장찌개 두국 두루치기 두부 두부구이 두부장국 두부조림 두부찌개 둑 드레싱 들깨 등 등갈비 등뼈 등심 딸기 땅 땅콩 땅콩강정 땅콩엿 떡 떡갈비 떡국 떡볶이 떡찜 라면 라이스 란국 란부 란스 란전 랑크 래미 랭 러스 레몬 렌치 렘 렛 로니 루츠 리 리소스 리알 리야 리회 링클 마늘 마늘빵 마약 마요 마요네즈 마카로니 마트 마파두부 막국수 만 만두 만두전골 말 말랭이 맛 맛살 맛살조림 

In [ ]:
# N-Gram 유용한 값 찾기
n_gram, tokeNgram = 2, []
def ngram(token, n):
    return [token[_: _+n] for _ in range(0, (len(token)-n+1))]

for _ in tokens:
    tokeNgram += ngram(_, n_gram)

from nltk import Text
ngramObj = Text(tokeNgram)   # [list] 객체로 Text 객체의 생성

In [ ]:
%matplotlib inline
from matplotlib import rc, rcParams
import matplotlib.pyplot as plt
rc('font', family=['NanumGothic','Malgun Gothic']) # 한글의 표시
rcParams['axes.unicode_minus'] = False             # '-' 표시의 처리
plt.figure(figsize=(15, 3))  # 파레트 설정
ngramObj.plot(70)

In [ ]:
ngramObj.dispersion_plot(["튀김","구이", "얼큰"])

In [ ]:
# itemIndexTemp : NER 객체어 사전
valid_token = []
unvalid_token = []
voca_tokens = ngramObj.vocab().most_common()

from tqdm import tqdm
for _ in tqdm(voca_tokens):
    if _[0] in itemIndexTemp: valid_token.append(_[0])
    else: unvalid_token.append(_[0])

# 유효하게 추출된 단어는 96개
# 나머지 5458개로 작업을 시작하기
# [_ for _ in itemIndexTemp if _ == "매실"]
len(valid_token), len(unvalid_token)

## **2 네이버 블로그 내용 수집하기**
블로그 URL 에서 본문을 수집 분석하기

In [ ]:
# iframe 으로 본문 url 찾은 뒤, //div[@id="postListBody"] 내용의 출력
import requests
from lxml.html import fromstring

def get_blog_post(url):
    userAgent   = {"user-agent":"Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0"}
    rsp         = requests.get(url, headers=userAgent)
    rsp_lxml    = fromstring(rsp.text) # './/태그[contains(@속성명, "속성값")]'
    text_result = rsp_lxml.xpath('.//iframe[@id="mainFrame"]/@src')
    get_url     = "https://blog.naver.com" + text_result[0]
    resp        = requests.get(get_url)
    resp_lxml   = fromstring(resp.text)
    blog_xpath  = '//div[@id="postListBody"]//table//text()'
    return " ".join([_.strip() for _ in resp_lxml.xpath(blog_xpath) if _.strip()])

url = "https://blog.naver.com/k77609/221366368354"
url = 'https://blog.naver.com/0167513714/221616708943'
url = 'https://blog.naver.com/hnn888/221411518781'
blog_text = get_blog_post(url)
blog_text[:250]

In [ ]:
from konlpy.tag import Okt, Mecab
tokens = [_[0]+"/"+_[1]  for _ in Okt().pos(blog_text, stem=True)]

from nltk import Text
token_blog = Text(tokens)   # [list] 객체로 Text 객체의 생성

In [ ]:
%matplotlib inline
from matplotlib import rc, rcParams
import matplotlib.pyplot as plt
rc('font', family=['NanumGothic','Malgun Gothic']) # 한글의 표시
rcParams['axes.unicode_minus'] = False             # '-' 표시의 처리
plt.figure(figsize=(15, 3))  # 파레트 설정
token_blog.plot(50)

In [ ]:
token_blog.vocab().most_common(5)
# Mecab().pos(blog_text)
# for _ in Okt().pos(blog_text, stem=True):
#     if _[1] not in ['Punctuation', "Josa",\
#         "Determiner", "Number", "Foreign", "Conjunction"]:
#         print (_[0], _[1])
#         break

## **1 음식의 Tf-idf 가중치 적용하기**
1. 음식목록 등 자기가 찾고자 하는 Theme 의 대상들을 특정한 뒤,
1. 여기서 수집된 단어를 대상으로 가중치값 비교하기

## **2 청심중고교 메뉴 데이터 호출**
무영 메뉴작업 목록 호출하기
```python
from muyong.food import menu_chungsim
from tqdm import tqdm
date   = [str(_).split(" ")[0][:-3]  for _ in pd.date_range(start='1/1/2017', end="8/1/2019", freq='MS')]
result = [menu_chungsim(_).values.tolist()  for _ in tqdm(date)]
temp   = []
for _ in result:
    temp.extend(_)
temp = pd.DataFrame(temp, columns = ["Date", "조식", "중식", "석식"])
```

In [ ]:
# 2017-01-01 부터 수집 가능합니다.
import pandas as pd
df = pd.read_csv('data/menu_chungsim.csv')
df.head(2)

In [ ]:
import pandas as pd
df = pd.read_csv('data/menu_chungsim.csv')
df.loc[0]['석식']

## **3 성균관 대학교 메뉴 데이터 호출**
무영 메뉴작업 목록 호출하기 : 1년 전까지만 호출가능
```python
div class="weekly_list"
div class="tabCon listcon1"
div class="weeListWrap"   # div class="weeListTit" : 날짜만 추출시
idx_date_range = [_.date().strftime("%Y-%m-%d")  
    for _ in pd.date_range(start=idx_date, periods=len(menu_detail))]
```